In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt 
from scipy.stats import norm 

In [12]:
ticker = [ 'AAPL', 'MSFT', 'NVDA', 'AMZN', 'TSLA','JPM', 'GS', 'V', 'BAC', 'JNJ', 'PFE', 'PG', 'KO',
          'XOM', 'CVX', 'CAT', 'TLT','BND','GLD', 'SPY']
weights = np.array([1/len(ticker)] * len(ticker))
initial_asset = 10000000

data = yf.download(ticker, start='2010-01-01', end='2026-01-01')
returns = data.pct_change().dropna()

[*********************100%***********************]  20 of 20 completed


In [8]:
avg_returns = returns.mean()
cov_matrix = returns.cov()

print(avg_returns)


Price   Ticker
Close   AAPL      0.001089
        GLD       0.000369
        SPY       0.000577
        TLT       0.000152
High    AAPL      0.001047
        GLD       0.000364
        SPY       0.000555
        TLT       0.000146
Low     AAPL      0.001084
        GLD       0.000368
        SPY       0.000572
        TLT       0.000144
Open    AAPL      0.001105
        GLD       0.000368
        SPY       0.000577
        TLT       0.000149
Volume  AAPL      0.051841
        GLD       0.105805
        SPY       0.051970
        TLT       0.088000
dtype: float64


In [13]:
n_simulations = 10000
days = 1

L = np.linalg.cholesky(cov_matrix)

portfolio_simulations = np.zeros(n_simulations)

for i in range (n_simulations):
    random_noise = np.random.normal(size=(L.shape[1], days))
    simulated_return = avg_returns.values.reshape(-1,1) + np.dot(L, random_noise)

    portfolio_returns = np.dot(weights, simulated_return)
    portfolio_simulations[i] = initial_asset * (1+portfolio_returns)

C:\Users\phoeb\AppData\Local\Temp\ipykernel_33900\3552679164.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  portfolio_simulations[i] = initial_asset * (1+portfolio_returns)


In [15]:
losses = initial_asset - portfolio_simulations
var_95 = np.percentile(losses, 95)
expected_shortfall = losses[losses> var_95].mean()

print(f"1-Day 95% VaR: ${var_95:,.2f}")
print(f"1-Day 95% Expected Shortfall: ${expected_shortfall:,.2f}")

1-Day 95% VaR: $850,616.77
1-Day 95% Expected Shortfall: $1,101,116.18
